In [ ]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=42)

In [ ]:
# target name 확인 - 어떤 뉴스 그룹이 있는지
news_data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
news_data.target

array([10,  3, 17, ...,  3,  1,  7])

In [ ]:
# 데이터 (뉴스 기사 확인)
print(news_data['data'][0])

From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!




In [ ]:
 # 훈련 데이터 가져오기. header, footers, 특수문자를 제거하고 불러오기
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=42)
X_train = train_news['data']
y_train = train_news['target']

test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'), random_state=42)
X_test = test_news['data']
y_test = test_news['target']

In [36]:
X_train[0]

'wondering anyone could enlighten car saw day 2 door sport car looked late 60 early 70 called bricklin door really small addition front bumper separate rest body know anyone tellme model name engine spec year production car made history whatever info funky looking car please e mail'

In [38]:
y_train

array([7, 4, 4, ..., 3, 1, 8])

# 텍스트 데이터 분석과 전처리

In [7]:
import pandas as pd
import numpy as np
import urllib.request

import matplotlib.pyplot as plt

In [11]:
# 뉴스 데이터의 기본 통계와 구조 파악
print(f'Number of training samples: {len(X_train)}')
print(f'Number of test samples: {len(X_test)}')

# 뉴스 데이터의 첫 번째 샘플 확인
print(X_train[0])

Number of training samples: 11314
Number of test samples: 7532
I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re

nltk.download('stopwords')
nltk.download('wordnet')

# 정규화, 불용어 제거, 표제어 추출 함수 정의
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # 소문자 변환
    text = text.lower()
    # 특수문자 제거
    text = re.sub(r'\W', ' ', text)
    # 불용어 제거 및 표제어 추출
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])

    return text

# 훈련 데이터와 테스트 데이터 전처리
X_train = [preprocess_text(text) for text in X_train]
X_test = [preprocess_text(text) for text in X_test]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
X_train[0]

'wondering anyone could enlighten car saw day 2 door sport car looked late 60 early 70 called bricklin door really small addition front bumper separate rest body know anyone tellme model name engine spec year production car made history whatever info funky looking car please e mail'

In [17]:
X_test[0]

'little confused model 88 89 bonnevilles heard le se lse sse ssei could someone tell difference far feature performance also curious know book value prefereably 89 model much le book value usually get word much demand time year heard mid spring early summer best time buy'

In [39]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


# 모델링

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [28]:
# 모델 선택 및 훈련

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# 하이퍼파라미터 튜닝

param_grid = {"C": [0.1,1,10]}
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)

best_model = grid_search.best_estimator_


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

# 평가



In [43]:
from sklearn.metrics import accuracy_score, classification_report, pair_confusion_matrix, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
# 예측
y_pred = best_model.predict(X_test_tfidf)

# 평가 지표 출력
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred, target_names=news_data.target_names))


Accuracy: 0.6550716941051513
                          precision    recall  f1-score   support

             alt.atheism       0.46      0.45      0.46       319
           comp.graphics       0.58      0.66      0.62       389
 comp.os.ms-windows.misc       0.62      0.59      0.60       394
comp.sys.ibm.pc.hardware       0.64      0.59      0.61       392
   comp.sys.mac.hardware       0.71      0.67      0.69       385
          comp.windows.x       0.78      0.68      0.73       395
            misc.forsale       0.76      0.77      0.76       390
               rec.autos       0.70      0.65      0.68       396
         rec.motorcycles       0.47      0.76      0.58       398
      rec.sport.baseball       0.78      0.80      0.79       397
        rec.sport.hockey       0.89      0.86      0.88       399
               sci.crypt       0.84      0.64      0.73       396
         sci.electronics       0.52      0.56      0.54       393
                 sci.med       0.71      0.74 

# 실제 뉴스 기사를 넣어서 카테고리를 잘 맞추는지 확인

In [54]:
def predict_news_category(article):
    """
    새로운 기사를 입력으로 받아서 예측된 카테고리를 반환하는 함수

    Parameters:
    article (str): 예측할 뉴스 기사 텍스트

    Returns:
    str: 예측된 카테고리 이름
    """
    # 기사 전처리
    article_preprocessed = preprocess_text(article)
    # 기사 벡터화
    article_tfidf = vectorizer.transform([article_preprocessed])
    # 카테고리 예측
    prediction = best_model.predict(article_tfidf)

    return news_data.target_names[prediction[0]]



In [58]:
# 실제 뉴스 기사 입력 및 예측
predicted_category = predict_news_category("North Korea start war with South Korea")

print(f'The predicted category is: {predicted_category}')


The predicted category is: talk.politics.misc


In [59]:
# 실제 뉴스 기사 입력 및 예측
predicted_category = predict_news_category("The New York Yankees secured a dramatic victory over the Boston Red Sox, winning 5-4 in the final inning with a walk-off home run by Aaron Judge.")

print(f'The predicted category is: {predicted_category}')

The predicted category is: rec.sport.baseball


In [60]:
# 실제 뉴스 기사 입력 및 예측
predicted_category = predict_news_category("Fuckin bacon Kim Jung eun")

print(f'The predicted category is: {predicted_category}')

The predicted category is: rec.motorcycles
